# 0] Imports

In [ ]:
!pip install transformers


In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast
from transformers import BertTokenizer, BertModel
import base64
import requests
import pandas as pd
import os
from transformers import AutoTokenizer

import nltk


In [ ]:
import torch
import torch.nn as nn
# specify GPU
device = torch.device("cuda")

# 1] Load Dataset

In [ ]:
data_url = 'https://raw.githubusercontent.com/shammur/SemEval2022Task3/main/data/train/train_subtask-1/en/'
folds = [pd.read_csv(os.path.join(data_url, 'En-Subtask1-fold_{}.tsv'.format(i)), sep='\t') \
         for i in range(3)] #reading all folds from GitHub

df = pd.concat(folds[0:3]) #concatinate all folds in 1 fold (only 2 folds because 3rd used as test)
print(df)
print(len(df))

In [ ]:
df.shape

In [ ]:
# check class distribution
df['Labels'].value_counts(normalize = True)

In [ ]:
df=df.sort_values(['ID'])


In [ ]:
df

In [ ]:
test_df_patterns=pd.concat([df[1:451], df[455:508],df[511:701], df[2657:3291], df[32977:3342], df[4330:4508],df[5393:5437], df[5615:]])

In [ ]:
test_df_patterns

In [ ]:
len(test_df_patterns)

In [ ]:
new_df=df[~df.ID.isin(test_df_patterns.ID)]

In [ ]:
new_df

In [ ]:
len(df)

In [ ]:
left_test_to_be_added= len(df)*0.7 - len(test_df_patterns) 

In [ ]:
left_test_to_be_added

In [ ]:
percentage_test= left_test_to_be_added * 100/ len(new_df) 

In [ ]:
percentage_test /100

###1.1] TF-IDF : extract 2 entities

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
ngram=1
# #converting to lower case
df_en2=df
df_en2['Sentence'] = df_en2.Sentence.map(lambda x: x.lower()) 
#Removing the punctuation
df_en2['Sentence'] = df_en2.Sentence.str.replace('[^\w\s]', '')  

#splitting the word into tokens
#man_train_set_df['Sentence'] = man_train_set_df['Sentence'].apply(tokenize.word_tokenize) 
en_texts=df_en2['Sentence']

tfidf_vectorizer = TfidfVectorizer(encoding='utf-8', lowercase=True, ngram_range=(1, ngram), norm='l1', 
                                   use_idf=True, max_df=0.05)#, min_df=3)# max_features=1000) 

In [ ]:
en_feat = tfidf_vectorizer.fit_transform(en_texts)

In [ ]:
en_feat.shape

In [ ]:
TFIDF_Dict=[word for word in set([ell for lis in [el.split() for el in en_texts.tolist()] for ell in lis]) if word not in tfidf_vectorizer.get_feature_names()]

In [ ]:
TFIDF_Dict

In [ ]:
TFIDF_vocab=dict(zip(tfidf_vectorizer.get_feature_names(), en_feat.toarray()[0])) #key= word, value=vector (cat, dog)

In [ ]:
TFIDF_vocab=[w for w in TFIDF_vocab]
#TFIDF_vocab

In [ ]:
df = pd.concat(folds[0:3]) #concatinate all folds in 1 fold (only 2 folds because 3rd used as test)

##1.2] Dataset Split

In [ ]:
df_train_val, df_test = train_test_split(new_df, shuffle=True, 
                                                                    test_size=percentage_test /100, 
                                                                    stratify=new_df['Labels'])

# we will use temp_text and temp_labels to create validation and test set
df_train, df_val  = train_test_split(df_train_val, 
                                                                shuffle=True, 
                                                                test_size=0.1, 
                                                                stratify=df_train_val['Labels'])

In [ ]:
len(df_train_val)

In [ ]:
len(df_test)

In [ ]:
len(df_train)

In [ ]:
len(df_val)

In [ ]:
df_test=pd.concat([df_test,test_df_patterns])


In [ ]:
df_test

In [ ]:
df_train=df_train.set_index('ID')
df_train

In [ ]:
df_test=df_test.set_index('ID')
df_test

In [ ]:
df_val=df_val.set_index('ID')
df_val

## 1.3] Add 2 nouns with sep tag and append to sentence

In [ ]:
def addNouns(df,TFIDF_vocab):
  nouns=""
  first=1
  for word in df['Sentence'].split():
    #print('word', word)
    if word in TFIDF_vocab:
      #print('yes in TF-IDF')
      nouns=" [SEP] " if first==1 else nouns
      nouns+=", "+ word
      first=0
  return nouns

In [ ]:
df_test['nouns']=df_test.apply(addNouns, axis=1, args=(TFIDF_vocab,))
df_test['Sentence']=df_test['Sentence']+df_test['nouns']
del df_test['nouns']
df_test

In [ ]:
df_train['nouns']=df_train.apply(addNouns, axis=1, args=(TFIDF_vocab,))
df_train['Sentence']=df_train['Sentence']+df_train['nouns']
del df_train['nouns']
df_train

In [ ]:
df_val['nouns']=df_val.apply(addNouns, axis=1, args=(TFIDF_vocab,))
df_val['Sentence']=df_val['Sentence']+df_val['nouns']
del df_val['nouns']
df_val

# Example

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

example_text = 'I will watch Memento tonight. This is a 2nd sentence. Third'
bert_input = tokenizer(example_text,padding='max_length', max_length = 20, 
                       truncation=True, return_tensors="pt")

In [ ]:
print(bert_input['input_ids'])
print(bert_input['token_type_ids'])
print(bert_input['attention_mask'])

In [ ]:
example_text = tokenizer.decode(bert_input.input_ids[0])

print(example_text)

# Dataset class

In [ ]:
import torch
import numpy as np
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {0:0,
          1:1
          }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['Labels']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['Sentence']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

# Bert Model

In [ ]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

# Train

In [ ]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  
EPOCHS = 10
model = BertClassifier()
LR = 1e-6
              
train(model, df_train, df_val, LR, EPOCHS)

# Test

In [ ]:
test = Dataset(df_test)

test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

if use_cuda: model = model.cuda()
outputs=[]
total_acc_test = 0
with torch.no_grad():

  for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)
              outputs.extend(output)
              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
print(f'Test Accuracy: {total_acc_test / len(df_test): .3f}')

In [ ]:
#outputs

In [ ]:
len(outputs)

In [ ]:
len(df_test)

In [ ]:
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

y_dev_pred = [int(el[1] > el[0]) for el in outputs]

precision_recall_fscore_support(df_test['Labels'],y_dev_pred , average='macro')

In [ ]:
precision_recall_fscore_support(df_test['Labels'],y_dev_pred , average='binary')

In [ ]:
df_test['y_dev_pred'] = y_dev_pred 
df_test[(df_test['Labels'] == 1) & (df_test['y_dev_pred'] == 1)]

df_testWrong=df_test[(df_test['Labels'] != df_test['y_dev_pred'])]
df_testWrong

count=0
for s in df_testWrong['Sentence']:
    #print(s)
    if 'specifically' in s:
        count+=1
print(count)